In [102]:
from config import config_erkan, config_halit
from binance.um_futures import UMFutures
import ta
import pandas as pd
from time import sleep
from binance.error import ClientError
import logging
from binance.client import Client
from binance.exceptions import BinanceAPIException
import pandas as pd
import time
from datetime import datetime

In [103]:
client = UMFutures(key = config_erkan['key'], secret=config_erkan['secret'])

In [104]:
tp = 0.012
sl = 0.009
volume = 10  # volume for one order (if its 10 and leverage is 10, then you put 1 usdt to one position)
leverage = 1
type = 'ISOLATED'  # type is 'ISOLATED' or 'CROSS'
qty = 5

In [152]:
# getting your futures balance in USDT
def get_balance_usdt():
    try:
        response = client.balance(recvWindow=6000)
        for elem in response:
            if float(elem['balance']) > 0:
                print(elem['asset'],":", float(elem['balance']))

    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )

print(get_balance_usdt())

USDT : 756.89972241
None


In [92]:
def get_tickers_usdt():
    tickers = []
    resp = client.ticker_price()
    for elem in resp:
        if 'USDT' in elem['symbol']:
            tickers.append(elem['symbol'])
    return tickers

# Getting candles for the needed symbol, its a dataframe with 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'
def klines(symbol):
    try:
        resp = pd.DataFrame(client.klines(symbol, '1h'))
        resp = resp.iloc[:,:6]
        resp.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
        resp = resp.set_index('Time')
        resp.index = pd.to_datetime(resp.index, unit = 'ms')
        resp = resp.astype(float)
        return resp
    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )


# Set leverage for the needed symbol. You need this bcz different symbols can have different leverage
def set_leverage(symbol, level):
    try:
        response = client.change_leverage(
            symbol=symbol, leverage=level, recvWindow=6000
        )
        print(response)
    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )

# The same for the margin type
def set_mode(symbol, type):
    try:
        response = client.change_margin_type(
            symbol=symbol, marginType=type, recvWindow=6000
        )
        print(response)
    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )

# Price precision. BTC has 1, XRP has 4
def get_price_precision(symbol):
    resp = client.exchange_info()['symbols']
    for elem in resp:
        if elem['symbol'] == symbol:
            return elem['pricePrecision']


# Amount precision. BTC has 3, XRP has 1
def get_qty_precision(symbol):
    resp = client.exchange_info()['symbols']
    for elem in resp:
        if elem['symbol'] == symbol:
            return elem['quantityPrecision']
        




In [144]:
data = klines('BTCUSDT')
print(data)

                        Open     High      Low    Close    Volume
Time                                                             
2024-11-04 03:00:00  69288.7  69495.3  68980.0  69034.0  7804.917
2024-11-04 04:00:00  69034.0  69397.2  69033.9  69143.4  4307.713
2024-11-04 05:00:00  69143.3  69174.0  68713.4  68811.4  6191.590
2024-11-04 06:00:00  68811.4  69159.3  68808.1  69115.6  4747.282
2024-11-04 07:00:00  69115.5  69166.0  68600.9  68601.0  6073.573
...                      ...      ...      ...      ...       ...
2024-11-24 18:00:00  95992.8  96633.0  95913.0  96605.7  5150.949
2024-11-24 19:00:00  96605.7  96605.7  96200.0  96396.8  3311.507
2024-11-24 20:00:00  96396.8  96900.0  96265.1  96888.0  4246.771
2024-11-24 21:00:00  96888.0  97061.4  96645.3  97037.7  3708.058
2024-11-24 22:00:00  97037.7  97890.0  97037.7  97498.3  7024.389

[500 rows x 5 columns]


In [95]:
# Your current positions (returns the symbols list):
def get_pos():
    try:
        resp = client.get_position_risk()
        pos = []
        for elem in resp:
            if float(elem['positionAmt']) != 0:
                pos.append(elem['symbol'])
        return pos
    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )

def check_orders():
    try:
        response = client.get_orders(recvWindow=6000)
        sym = []
        for elem in response:
            sym.append(elem['symbol'])
        return sym
    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )

# Close open orders for the needed symbol. If one stop order is executed and another one is still there
def close_open_orders(symbol):
    try:
        response = client.cancel_open_orders(symbol=symbol, recvWindow=6000)
        print(response)
    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )

In [94]:
orders = 0
symbol = ''
# getting all symbols from Binace Futures list:
symbols = get_tickers_usdt()

In [146]:
pos = get_pos()
pos

['EIGENUSDT',
 'RDNTUSDT',
 'HIGHUSDT',
 'OPUSDT',
 'CHRUSDT',
 'STMXUSDT',
 'KDAUSDT',
 'NMRUSDT',
 'ENSUSDT',
 'ACTUSDT',
 'STXUSDT',
 'ZETAUSDT',
 'RONINUSDT',
 'XVSUSDT',
 'VOXELUSDT',
 'MAVUSDT',
 'MEMEUSDT',
 'CHESSUSDT',
 'NEOUSDT',
 'MINAUSDT',
 'NULSUSDT',
 'NOTUSDT',
 'CKBUSDT',
 'TLMUSDT',
 'AAVEUSDT',
 '1000LUNCUSDT',
 'LEVERUSDT',
 'AMBUSDT',
 'CTSIUSDT',
 'SUNUSDT']

In [148]:
ord = check_orders()
ord

['AAVEUSDT',
 'AAVEUSDT',
 'KDAUSDT',
 'KDAUSDT',
 'EIGENUSDT',
 'EIGENUSDT',
 'HIGHUSDT',
 'HIGHUSDT',
 'MINAUSDT',
 'MINAUSDT',
 'CTSIUSDT',
 'CTSIUSDT',
 'ENSUSDT',
 'ENSUSDT',
 'NEOUSDT',
 'NEOUSDT',
 'STXUSDT',
 'STXUSDT',
 'MEMEUSDT',
 'MEMEUSDT',
 'CKBUSDT',
 'CKBUSDT',
 'ACTUSDT',
 'ACTUSDT',
 'ACTUSDT',
 'ACTUSDT',
 'ACTUSDT',
 'TLMUSDT',
 'TLMUSDT',
 'AMBUSDT',
 'AMBUSDT',
 'LEVERUSDT',
 'LEVERUSDT',
 'CHRUSDT',
 'CHRUSDT',
 'RDNTUSDT',
 'RDNTUSDT',
 'XVSUSDT',
 'XVSUSDT',
 'STMXUSDT',
 'STMXUSDT',
 'OPUSDT',
 'OPUSDT',
 'VOXELUSDT',
 'VOXELUSDT',
 'NMRUSDT',
 'NMRUSDT',
 'SUNUSDT',
 'SUNUSDT',
 'MAVUSDT',
 'MAVUSDT',
 'NULSUSDT',
 'NULSUSDT',
 'CHESSUSDT',
 'CHESSUSDT',
 'ZETAUSDT',
 'ZETAUSDT',
 'RONINUSDT',
 'RONINUSDT',
 '1000LUNCUSDT',
 '1000LUNCUSDT']